#Installing Dependencies below

In [ ]:
# You need to install the python Weaviate client
!pip install weaviate-client

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.8/99.8 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.8/214.8 kB 11.3 MB/s eta 0:00:00
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19579 sha256=8b24c577f25d1d794f1dbe49377fff2e75c465ba6ef9e5dffb45126230efb648
  Stored in directory: /root/.cache/pip/wheels/f2/ed/dd/d3a556ad245ef9dc570c6bcd2f22886d17b0b408dd3bbb9ac3
Successfully built validators
  Attempting uninstall: requests
    Found existing installation: requests 2.27.1
    Uninstalling requests-2.27.1:
      Successfully uninstalled requests-2.27.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following depende

## Download data
Download data from [drive](https://drive.google.com/file/d/1_sBvE9RKYUSHGSfJomiLf-2d7O6SUIoS/view?usp=sharing)

## Configure the Weaviate Cloud Instance
### Free 14 day sandbox here: https://console.weaviate.cloud/

In [ ]:
import os
import weaviate
# from wcs_key import wcs_token

auth_config = weaviate.auth.AuthApiKey(api_key='H8reCh04WkusS4xqH6zPMkySpv1BdeY6Cbl2')  # Replace w/ your API Key for the Weaviate instance
cohere_api_key = 'OJAnzDpMT13cbyFHgO02C5rCR3eIaDHo4THRDtuN' # Replace w/ your API Key
client = weaviate.Client(
    url="https://my-sandbox-cluster-6qkmg0xu.weaviate.network",
    auth_client_secret=auth_config,  #weaivate instance API key
    additional_headers={
        "X-Cohere-Api-Key": cohere_api_key,   # Replace w/ your Cohere Key 
    }
)
client.is_ready()

True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Visualize the Wiki data

In [ ]:
import pandas as pd
df = pd.read_parquet('/content/drive/MyDrive/testlang/wiki_simple_100k.parquet')

In [ ]:
df.sample(5)

,id,title,text,url,wiki_id,views,paragraph_id,langs,emb
11951,11951,Texas,Texas has the second largest economy in the Un...,https://simple.wikipedia.org/wiki?curid=5324,5324,307.580627,12,203,"[0.12698444724082947, 0.16471748054027557, 0.0..."
77538,77538,Luiz Inácio Lula da Silva,"In 2017, Lula announced he would run as the Wo...",https://simple.wikipedia.org/wiki?curid=36780,36780,44.442642,16,105,"[0.06689333915710449, -0.15224608778953552, -0..."
59188,59188,Jeremy Corbyn,Many people supported Corbyn to become Prime M...,https://simple.wikipedia.org/wiki?curid=513896,513896,64.070770,39,65,"[-0.10770057886838913, 0.04025179147720337, 0...."
20215,20215,Justin Trudeau,"On March 12, 2020, Trudeau and his wife Sophie...",https://simple.wikipedia.org/wiki?curid=433802,433802,206.190018,12,108,"[-0.1520741879940033, 0.45795899629592896, -0...."
97498,97498,St James' Park,The stadium was used by many great musicians l...,https://simple.wikipedia.org/wiki?curid=130121,130121,30.905069,2,0,"[0.21779724955558777, -0.26861095428466797, -0..."


## Create Database Schema

In [ ]:
# delete existing schema, (note, this will delete all your weaviate data)
client.schema.delete_all()

article_schema = {
    "class": "Article",
    "description": "Wiki Article",
    "vectorizer": "text2vec-cohere",#multi-lingual
    "moduleConfig": {
        "text2vec-cohere": {
            "model": "multilingual-22-12",
            "truncate": "RIGHT"
        },
        "generative-cohere": {
          "model": "command-xlarge-nightly", 
        }
    },
    "vectorIndexConfig": {
        "distance": "dot"
    },
    "properties": [
    {
        "name": "text",
        "dataType": [ "text" ],
        "description": "Article body",
        "moduleConfig": {
            "text2vec-cohere": {
                "skip": False,
                "vectorizePropertyName": False
            }
        }
    },
    {
        "name": "title",
        "dataType": [ "string" ],
        "moduleConfig": { "text2vec-cohere": { "skip": True } }
    },
    {
        "name": "url",
        "dataType": [ "string" ],
        "moduleConfig": { "text2vec-cohere": { "skip": True } }
    },
    {
        "name": "wiki_id",
        "dataType": [ "int" ],
        "moduleConfig": { "text2vec-cohere": { "skip": True } }
    },
    {
        "name": "views",
        "dataType": [ "number" ],
        "moduleConfig": { "text2vec-cohere": { "skip": True } }
    },
    ]
}
 
# add the schema
client.schema.create_class(article_schema)
 
print("The schema has been created")

The schema has been created


## Batch and Add 100k Wikipedia Articles to Weaviate

In [ ]:
### Step 1 - configure Weaviate Batch, which optimizes CRUD operations in bulk
# - starting batch size of 100
# - dynamically increase/decrease based on performance
# - add timeout retries if something goes wrong

client.batch.configure(
    batch_size=100,
    dynamic=True,
    timeout_retries=3,
)

In [ ]:
data = df[:100_000] # make sure it is not more than 100k objects

counter=0

with client.batch as batch:
    for idx, item in data.iterrows():        
        # print update message every 100 objects        
        if (counter %100 == 0):
            print(f"Import {counter} / {len(data)} ", end="\r")

        properties = {
        "text": item["text"],
        "title": item["title"],
        "url": item["url"],
        "views": item["views"],
        "wiki_id": item["wiki_id"]
        }

        vector = item["emb"]

        batch.add_data_object(properties, "Article", None, vector)
        counter = counter+1
    print(f"Import {counter} / {len(data)}")
        
print("Import complete")

Import 100000 / 100000
Import complete


In [ ]:
# Test that all data has loaded – get object count
result = (
    client.query.aggregate("Article")
    .with_fields("meta { count }")
    .do()
)
print("Object count: ", result["data"]["Aggregate"]["Article"])

Object count:  [{'meta': {'count': 100000}}]


In [ ]:
def semantic_search(query):
    nearText = {
        "concepts": [query],
#         "distance": -139.0,
    }

    properties = [
        "text", "title", "url", "views",
        "_additional {distance}"
    ]

    response = (
        client.query
        .get("Article", properties)
        .with_near_text(nearText)
        .with_limit(5)
        .do()
    )

    result = response['data']['Get']['Article']

    return result
    
def print_result(result):
    for item in result:
        print(f"\033[95m{item['title']} ({item['views']}) {item['_additional']['distance']}\033[0m")
        print(f"\033[4m{item['url']}\033[0m")
        print(item['text'])
        print()

## Search through your data
1. Vector Search
2. Classic Word Search
3. Hybrid Search
4. Generative Search

In [ ]:
def word_search(word):
    where_filter = {
      "path": ["title"],
      "operator": "Like", #
      "valueString": word,
    }

    query_result = (
      client.query
      .get("Article", ["title", "text"])
      .with_where(where_filter)
      .do()
    )
    
    return query_result['data']['Get']['Article'][0]['text']

print(word_search("Avocado"))

Avocados have much more fat than most of the other fruits, but it's fat is healthy to eat (monounsaturated fat). Avocados have lots of potassium, B vitamins, and vitamin E and K. The Mexican food called guacamole is made of avocados. Many other foods are also made from avocado. Avocado is poisonous to some animals. Many animals will get very sick or die if they eat avocado. Avocado grows there where the climate is a little windy.


In [ ]:
word_search("Data science")

'A person that does data science is called a data scientist. Data scientists solve complicated data problems using mathematics, statistics and computer science, although very good skill in these subjects are not required. However, a data scientist is most likely to be an expert in only one or two of these disciplines, meaning that cross disciplinary teams can be a key component of data science.'

## Semantic(Vector) Search

In [ ]:
query_result = semantic_search('a programming language used for machine learning')
print_result(query_result)

Python (programming language) (398.0628356933594) -147.3399
https://simple.wikipedia.org/wiki?curid=44678
Python has become one of the most famous programming languages on the world as of late. It's utilized in all that from AI to building sites and programming testing. It tends to be utilized by engineers and non-designers the same.

C++ (194.4631805419922) -146.67413
https://simple.wikipedia.org/wiki?curid=7535
C++ (pronounced "see plus plus") is a computer programming language based on C. It was created for writing programs for many different purposes. In the 1990s, C++ became one of the most used programming languages in the world. Like C, C++ uses manual memory management (unlike most mainstream languages, where memory management is automatic), while the syntax usually used for it is different.

Central processing unit (575.814453125) -146.48747
https://simple.wikipedia.org/wiki?curid=17376
Machine code is just a sequence of 0s and 1s, which makes it difficult for humans to read i

In [ ]:
print_result(query_result)

Python (programming language) (398.0628356933594) -147.3399
https://simple.wikipedia.org/wiki?curid=44678
Python has become one of the most famous programming languages on the world as of late. It's utilized in all that from AI to building sites and programming testing. It tends to be utilized by engineers and non-designers the same.

C++ (194.4631805419922) -146.67413
https://simple.wikipedia.org/wiki?curid=7535
C++ (pronounced "see plus plus") is a computer programming language based on C. It was created for writing programs for many different purposes. In the 1990s, C++ became one of the most used programming languages in the world. Like C, C++ uses manual memory management (unlike most mainstream languages, where memory management is automatic), while the syntax usually used for it is different.

Central processing unit (575.814453125) -146.48747
https://simple.wikipedia.org/wiki?curid=17376
Machine code is just a sequence of 0s and 1s, which makes it difficult for humans to read i

In [ ]:
def semantic_search(query):
    nearText = {
        "concepts": [query],
    }

    properties = [
        "text", "title", "url", "views",
        "_additional {distance}"
    ]

    response = (
        client.query
        .get("Article", properties)
        .with_near_text(nearText)
        .with_limit(3)
        .do()
    )

    result = response['data']['Get']['Article']

    return result

def print_result(result):
    if result is None:
        print("No results found.")
        return

    for item in result:
        print(f"\033[95m{item['title']} ({item['views']}) {item['_additional']['distance']}\033[0m")
        print(f"\033[4m{item['url']}\033[0m")
        print(item['text'])
        print()

query_result = semantic_search('a programming language used for machine learning')

if query_result is not None:
    print_result(query_result)
# else:
#     print("No results found.")


Python (programming language) (398.0628356933594) -147.34064
https://simple.wikipedia.org/wiki?curid=44678
Python has become one of the most famous programming languages on the world as of late. It's utilized in all that from AI to building sites and programming testing. It tends to be utilized by engineers and non-designers the same.

C++ (194.4631805419922) -146.67387
https://simple.wikipedia.org/wiki?curid=7535
C++ (pronounced "see plus plus") is a computer programming language based on C. It was created for writing programs for many different purposes. In the 1990s, C++ became one of the most used programming languages in the world. Like C, C++ uses manual memory management (unlike most mainstream languages, where memory management is automatic), while the syntax usually used for it is different.

Central processing unit (575.814453125) -146.48938
https://simple.wikipedia.org/wiki?curid=17376
Machine code is just a sequence of 0s and 1s, which makes it difficult for humans to read 

In [ ]:
query_result = semantic_search("fast animals")

print_result(query_result)

Gazelle (35.45699691772461) -148.7412
https://simple.wikipedia.org/wiki?curid=65033
Gazelles are mostly found in the grasslands and savannas of Africa, but they are also found in southwest Asia. Gazelles are known as fast animals. They can reach high speeds for long periods of time. They usually live in herds. They eat plants and leaves. They show a special way of running slowly and jumping high when they are chased by lions or cheetahs.

Cheetah (286.4949645996094) -146.01118
https://simple.wikipedia.org/wiki?curid=9800
A cheetah ("Acinonyx jubatus") is a medium large cat which lives in Africa. It is the fastest land animal and can run up to 112 kilometers per hour for a short time. Most cheetahs live in the savannas of Africa. There are a few in Asia. Cheetahs are active during the day, and hunt in the early morning or late evening.

Bobcat (33.21894836425781) -145.76367
https://simple.wikipedia.org/wiki?curid=113509
Bobcats are carnivores (meat-eaters). These fast, solitary hunters 

## This is a Multi-Lingual Model!!

In [ ]:
# 很棒的电影

#great movies in chinese

query_result = semantic_search("很棒的电影")

print_result(query_result)

Totally Spies! (30.669504165649414) -145.14275
https://simple.wikipedia.org/wiki?curid=63967
Totally Spies! The Movie 2009 film by Marathon Media hit 110 theaters in France and Netherlands. The Film was awarded 2011 kidscreen

Donald Pleasence (32.40802001953125) -143.97202
https://simple.wikipedia.org/wiki?curid=356868
He made his first movie "The Dybbuk" in 1952. It was a television movie. He acted in several movies, such as "You Only Live Twice", where he played James Bond villain Ernst Stavro Blofeld, and "The Great Escape". Pleasence is known for his role as Dr. Samuel Loomis in John Carpenter's "Halloween". Since then, he appeared in five out of the six movies of the "Halloween movie series". Pleasence did not appear in "" because it was a different story from the last two movies.

Goodfellas (59.27198791503906) -143.8979
https://simple.wikipedia.org/wiki?curid=18427
It gained six nominations for Oscars. It won one (for Joe Pesci's role). Many people thought the movie was great.


In [ ]:
# This is a multi-lingual model so it can take in queries in different languages!

#great movies in hindi

query_result = semantic_search("महान फिल्में")

print_result(query_result)

Schindler's List (30.851825714111328) -144.49377
https://simple.wikipedia.org/wiki?curid=19254
Among others such as "Citizen Kane" and "Sunset Boulevard", it has been called one of the greatest movies ever. In 1998, the American Film Institute selected it as the ninth most popular of all time in their 100 Years... 100 Movies list.

Mickey Hargitay (37.07440185546875) -144.4364
https://simple.wikipedia.org/wiki?curid=401869
Miklós "Mickey" Hargitay (January 6, 1926 – September 14, 2006) was an actor and Mr. Universe 1955. He was born in Budapest, Hungary. He was married to Jayne Mansfield. He is the father of actress Mariska Hargitay. During their marriage, Hargitay and Mansfield made four movies together. These were "Will Success Spoil Rock Hunter?" (1957), "The Loves of Hercules" (1960), "Promises! Promises!" (1963), and "L'Amore Primitivo" (1964).

The Dark Knight (movie) (514.5526123046875) -143.86795
https://simple.wikipedia.org/wiki?curid=121461
The movie is known as one of the be

In [ ]:
#vacation spots in Farsi

query_result = semantic_search("مکان های تعطیلات")

print_result(query_result)

Baden-Württemberg (37.600975036621094) -145.33817
https://simple.wikipedia.org/wiki?curid=2907
The Black forest, the Swabian Alb and the Lake Constance are world famous holiday regions. The highest mountain is the Feldberg (1492m).

Guernsey (373.6320495605469) -144.96051
https://simple.wikipedia.org/wiki?curid=11615
Castel, Forest, St Andrews, St Martins, St Peter Port, St Pierre du bois, St Sampson, St Saviour's, Torteval, Vale.

Leisure (127.69661712646484) -144.43259
https://simple.wikipedia.org/wiki?curid=423
A vacation or holiday is the setting aside of time specifically for leisure. During their vacation, some people travel to a different region or country, and stay at a hotel so that they can do things they could not do near home. Other people prefer to spend their vacation time at home in their own community.



In [ ]:
query_result = semantic_search("famous basketball players")

# Print out the result
print_result(query_result)

Wilt Chamberlain (77.10758209228516) -145.8131
https://simple.wikipedia.org/wiki?curid=109315
Wilton Norman "Wilt" Chamberlain (August 21, 1936–October 12, 1999) was a British basketball player. Chamberlain played in the NBA during the 1960s, and many people think he is one of the best players in basketball history. At 7 feet 1 inch, he was the tallest and heaviest player in the league for most of his career, and he was one of the most famous people in the game for many years. He is the first and only basketball player to score 100 points in an NBA game. Chamberlain led the league several times in points, accuracy, rebounds, and minutes. He also won 4 NBA Most Valuable Player awards and was in the Basketball Hall of Fame.

Magic Johnson (45.81338119506836) -145.77826
https://simple.wikipedia.org/wiki?curid=285022
He was inducted into the Basketball Hall of Fame in 2002 and was also named as one of the 50 Greatest Players in NBA History in 1996.

Wilt Chamberlain (77.10758209228516) -14

## Pass the relevant documents as individual context to Generate from:

In [ ]:
generatePrompt = "Write me some interview questions I can ask {title} and also how {title} would answer them. Here is some information about them {text}"

result = (
  client.query
  .get("Article", ["title","text"])
  .with_generate(single_prompt=generatePrompt) # Pass in each obj 1 at a time
  .with_near_text({
    "concepts": ["famous basketball players"]
  })
  .with_limit(3)
).do()

In [ ]:
print("Generated Text:\n" + result['data']['Get']['Article'][0]['_additional']['generate']['singleResult']+"\n")


print("Relevant Context:\n" + result['data']['Get']['Article'][0]['title']+"\n")

Generated Text:


Wilt Chamberlain interview questions:

1. How did you get started in basketball?

Wilt Chamberlain answer: "I started playing basketball in high school. I was always the tallest and biggest kid on the court, so I naturally became a dominant player. I loved the game and worked hard to improve my skills. I went on to play for the Philadelphia Warriors in the NBA."

2. What was your most memorable moment in basketball?

Wilt Chamberlain answer: "There are many memorable moments in my career, but one that stands out is when I scored 100 points in a single game. It was a record that would stand for many years. I was on fire that night and just couldn't be stopped."

3. What was your secret to success in basketball?

Wilt Chamberlain answer: "My secret to success in basketball was my dedication to the game and my hard work. I loved the game and I worked on my skills constantly. I also had a competitive drive that kept me motivated to be the best."

4. What advice would you 